<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/jeevan_foodfinder_gemma_base_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook demonstrates the following:
* leverages a custom web-scraped dataset created by Jeevan from KaggleX
* fine tuning experiments on various gemma datasets

# Get access to Gemma via your Kaggle account:
* Log into your Kaggle account
* Request access to Gemma models using your Kaggle account. You can follow these instructions here: https://www.kaggle.com/code/nilaychauhan/get-started-with-gemma-using-kerasnlp
* You need to wait for confirmation. Note that this didn't take too long for me.
* Create an API key in your Kaggle account you will need later. You can follow these instructions here: https://christianjmills.com/posts/kaggle-obtain-api-key-tutorial/

# Ensure your Colab notebook can access Gemma:
* Add the Kaggle API key into your COLAB secrets. You can follow these instructions here: https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0

# Select an AI hardware accelerator
* Select hardware options near the top right of your Colab notebook
* I tested with A100 and it worked well. Note that I have a Colab Pro subscription.

# Install required packages

In [1]:
%%time
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3.3.3"

CPU times: user 31.5 ms, sys: 5.57 ms, total: 37.1 ms
Wall time: 4.92 s


# Import required packages


In [2]:
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
from keras import backend as K
import tensorflow
from IPython.display import Markdown
import textwrap
from google.colab import userdata
import json
import pandas as pd
import gc

# Configure this notebook
* set up KERAS parameters recommended by Google
* integrate KAGGLE API secret key

In [3]:
# set up KERAS parameters recommended by Google
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00" # Avoid memory fragmentation on JAX backend.

# integrate KAGGLE API secret key
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME') # Link to KAGGLE API secret key
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY') # Link to KAGGLE API secret key

# Retrieve the fine-tuning dataset
* You will need to first copy it to your gdrive
* Then you need to mount your gdrive on the left side panel in your colab notebook
* You will likely need to change the path to where you copied it in your gdrive

In [4]:
%%time
# make sure we can access the dataset
CSV_PATH="/content/drive/MyDrive/Kaggle_X/Jeevan_FoodFinder/qa_pairs_df.csv"
if not os.path.exists(CSV_PATH):
  raise Exception("Could not find the dataset.")
else:
  print("Found the dataset.")

# load it via pandas and summarize
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(CSV_PATH)
df.describe()

Found the dataset.
CPU times: user 13.7 ms, sys: 3.03 ms, total: 16.7 ms
Wall time: 18.5 ms


,question,answer
count,1121,1121
unique,717,772
top,What are the overall views of the customers on Sushi Zai in location?,"Sake (Nihonshu),Shochu (Japanese spirits)"
freq,5,39


# Define some useful functions

In [4]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

# Before fine-tuning, let's try various Gemma models
* using models listed here: https://keras.io/api/keras_nlp/models/gemma/gemma_backbone/#gemmabackbone-class
* note that not all models listed fit into 1 A100 GPU provided by colab


In [5]:
# Here is the prompt we will try
prompt = "What should I eat in Japan?"

# Load Gemma 2B base

In [7]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

CPU times: user 9.42 s, sys: 10.1 s, total: 19.5 s
Wall time: 48 s


# Test a simple query on Gemma 2B base

In [8]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)


CPU times: user 1min 8s, sys: 1 s, total: 1min 9s
Wall time: 35.5 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>This is a question that many people ask when they visit Japan. There are many different types of food in Japan, and it can be difficult to know what to eat. In this article, we will discuss some of the most popular foods in Japan and what you should eat in Japan.<br><br><h2>What is the most popular food in Japan?</h2><br><br>There are many different types of food in Japan, but some of the most popular include sushi, ramen, and okonomiyaki. Sushi is a type of Japanese cuisine that consists of rice and fish or other ingredients that have been rolled into a ball and then dipped in a sauce. Ramen is a type of noodle soup that is typically made with pork or chicken broth, noodles, and vegetables. Okonomiyaki is a type of Japanese pancake that is made with flour, eggs, and various other ingredients.<br><br><h2>What is the most popular drink in Japan?</h2><br><br>There are many different types of drinks in Japan, but some of the most popular include coffee, tea, and beer. Coffee is a popular drink in Japan, and there are many different types of coffee shops to choose from. Tea is also a popular drink in Japan, and there are many different types of tea to choose from. Beer is a popular drink in Japan, and there are many different types of beer to choose from.<br><br><h2>What is the most popular dessert in Japan?</h2><br><br>There are many different types of desserts in Japan, but some of the most popular include mochi, ice cream, and cheesecake. Mochi is a type of Japanese rice cake that is typically made with glutinous rice and is often served with ice cream or other toppings. Ice cream is a popular dessert in Japan, and there are many different types of ice cream to choose from. Cheesecake is a type of cake that is typically made with cream cheese and is often served with fruit or other toppings.<br><br><h2>What is the most popular food in Japan?</h2><br><br>There are many different types of food in Japan, but some of the most popular include sushi, ramen, and okonomiyaki. Sushi is a type of Japanese cuisine that consists of rice and fish or other ingredients that have been rolled into a ball and then dipped in a sauce. Ramen is a type of noodle soup that is typically made with pork or chicken broth, noodles, and vegetables. Okonomiyaki is a type of Japanese pancake that is made with flour, eggs, and various other ingredients.<br><br><h2>What is the most popular drink in Japan?</h2><br><br>There are many different types of drinks in Japan, but some of the most popular include coffee, tea, and beer. Coffee is a popular drink in Japan, and there are many different types of coffee shops to choose from. Tea is also a popular drink in Japan, and there are many different types of tea to choose from. Beer is a popular drink in Japan, and there are many different types of beer to choose from.<br><br><h2>What is the most popular dessert in Japan?</h2><br><br>There are many different types of desserts in Japan, but some of the most popular include mochi, ice cream, and cheesecake. Mochi is a type of Japanese rice cake that is typically made with glutinous rice and is often served with ice cream or other toppings. Ice cream is a popular dessert in Japan, and there are many different types of ice cream to choose from. Cheesecake is a type of cake that is typically made with cream cheese and is often served with fruit or other toppings.<br><br><h2>What is the most popular food in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular foods in the world include pizza, sushi, and tacos.<br><br><h2>What is the most popular drink in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular drinks in the world include coffee, tea, and beer.<br><br><h2>What is the most popular food in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular foods in the world include pizza, sushi, and tacos.<br><br><h2>What is the most popular drink in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular drinks in the world include coffee, tea, and beer.<br><br><h2>What is the most popular food in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular foods in the world include pizza, sushi, and tacos.<br><br><h2>What is the most popular drink in the world?</h2><br><br>There is no definitive answer to this question as it depends on personal preferences and cultural influences. However, some of the most popular drinks in the world include coffee, tea, and beer.<br><br><h2></blockquote></font>

# Load Gemma 2B instruct

In [9]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

CPU times: user 8.45 s, sys: 7.14 s, total: 15.6 s
Wall time: 16.3 s


# Test a simple query on Gemma 2B instruct

In [10]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 22.7 s, sys: 577 ms, total: 23.3 s
Wall time: 22.6 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>**Popular Japanese Cuisine:**<br><br>* **Sushi:** Raw fish served with rice and soy sauce.<br>* **Sashimi:** Raw fish served with soy sauce.<br>* **Tempura:** Deep-fried seafood or vegetables.<br>* **Yakitori:** Grilled skewers of chicken or vegetables.<br>* **Ramen:** Noodle soup with a variety of toppings.<br>* **Udon:** Noodle soup with wheat noodles.<br>* **Congee:** Noodle soup with rice noodles.<br>* **Sukiyaki:** A hot pot dish with meat, vegetables, and noodles.<br><br>**Other Traditional Japanese Dishes:**<br><br>* **Takoyaki:** Octopus balls in a savory sauce.<br>* **Kimchi:** Fermented cabbage with a spicy and sour flavor.<br>* **Unagi:** Grilled eel.<br>* **Tsukune:** Grilled pork belly.<br>* **Yakimochi:** Rice cakes filled with meat or seafood.<br><br>**Modern Japanese Cuisine:**<br><br>* **Sushi:** Modernized sushi with creative and innovative dishes.<br>* **Ramen:** Innovative ramen dishes with unique flavors and toppings.<br>* **Kaiten-zushi:** Sushi that is served on a plate with a variety of toppings.<br>* **Shabu-shabu:** Hot pot with simmering broth and vegetables.<br>* **Tempura:** Modernized tempura with a focus on quality and presentation.<br><br>**Regional Specialties:**<br><br>* **Okonomiyaki:** Savory pancake made with flour, eggs, and vegetables.<br>* **Kushikatsu:** Deep-fried skewers of meat or vegetables.<br>* **Yakisoba:** Noodles with a pork-based sauce.<br>* **Takoyaki:** A popular street food in Osaka and other cities.<br><br>**Tips for Eating in Japan:**<br><br>* **Be mindful of your portion sizes.** Japanese food is often very filling, so it's important to pace yourself.<br>* **Use chopsticks and a spoon together.** This is the traditional way to eat Japanese food.<br>* **Don't be afraid to ask for recommendations.** The staff in Japanese restaurants are often happy to help you choose a dish that you will enjoy.<br>* **Be aware of the hygiene standards in Japanese restaurants.** Food is handled very carefully, and you should always wash your hands before eating.<br>* **Enjoy the experience!** Eating in Japan is a unique and cultural experience, so relax and enjoy the food and the company.</blockquote></font>

# Load Gemma 1.1 2B instruct

In [11]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_2b_en")

CPU times: user 9.34 s, sys: 5.29 s, total: 14.6 s
Wall time: 40.9 s


# Test a simple query on Gemma 1.1 2B instruct

In [12]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 21.3 s, sys: 417 ms, total: 21.7 s
Wall time: 21.1 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>**Popular Dishes:**<br><br>* Sushi<br>* Ramen<br>* Tempura<br>* Soba<br>* Yakitori<br>* Tempura<br>* Okonomiyaki<br><br>**Regional Specialties:**<br><br>* Hokkaido: Seafood dishes<br>* Kansai: Okonomiyaki and takoyaki<br>* Okinawa: Okonomiyaki and sweet potato dishes<br>* Hiroshima: Hiroshima ramen<br>* Fukuoka: Hakata ramen<br>* Hokkaido: Miso soup<br><br>**Dietary Preferences:**<br><br>* Vegetarian: Tofu, vegetables, miso soup<br>* Vegan: Tofu, vegetables, seaweed, sweet potatoes<br><br>**Tips:**<br><br>* Explore local markets for fresh ingredients.<br>* Try street food for authentic experiences.<br>* Consider a set meal for convenience.<br>* Learn basic Japanese phrases for better dining experience.<br><br><br>**Additional Recommendations:**<br><br>* **Sushi:** Edomae sushi (raw sushi), nigiri sushi (rolled sushi), sashimi<br>* **Ramen:** Different regional variations, often spicy<br>* **Tempura:** Deep-fried seafood and vegetables<br>* **Okonomiyaki:** Savory pancake with various fillings<br>* **Yakitori:** Grilled skewers of meat and vegetables<br>* **Soba:** Wheat noodles with various toppings<br>* **Hiroshima Ramen:** Thick, flavorful broth with pork belly<br>* **Hakata Ramen:** Spicy and flavorful broth with pork belly<br><br><br>**Tips for Vegetarians:**<br><br>* Tofu is a staple ingredient in Japanese cuisine.<br>* Vegetables are widely available and delicious.<br>* Miso soup is a hearty and flavorful vegetarian dish.<br>* Tofu dishes are often seasoned with soy sauce, ginger, and garlic.<br><br><br>**Additional Tips:**<br><br>* Drink Japanese tea or coffee with meals.<br>* Use chopsticks to eat.<br>* Learn basic Japanese etiquette before dining.</blockquote></font>

# Load Gemma 2 2B base

In [13]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")

CPU times: user 9.82 s, sys: 9.2 s, total: 19 s
Wall time: 47.5 s


# Test a simple query on Gemma 2 2B base

In [14]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 11s, sys: 1.03 s, total: 2min 12s
Wall time: 1min


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Japan in a few weeks and I'm wondering what I should eat while I'm there. I'm not a big fan of sushi, but I'm sure I'll eat it. I'm also not a big fan of ramen, but I'm sure I'll eat it. I'm also not a big fan of curry, but I'm sure I'll eat it. I'm also not a big fan of tempura, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I'm sure I'll eat it. I'm also not a big fan of takoyaki, but I'm sure I'll eat it. I'm also not a big fan of gyoza, but I'm sure I'll eat it. I'm also not a big fan of yakitori, but I'm sure I'll eat it. I'm also not a big fan of okonomiyaki, but I</blockquote></font>

# Load Gemma 2 2B instruct

In [15]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

CPU times: user 9.43 s, sys: 7.44 s, total: 16.9 s
Wall time: 46.6 s


# Test a simple query on Gemma 2 2B instruct

In [16]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 41.6 s, sys: 848 ms, total: 42.4 s
Wall time: 41.1 s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>Japan is a culinary paradise, offering a diverse range of dishes to tantalize your taste buds. Here's a breakdown of must-try foods:<br><br>**Sushi & Sashimi:**<br><br>* **Nigiri:** Hand-formed rice topped with fresh fish or seafood.<br>* **Sashimi:** Thinly sliced raw fish served without rice.<br>* **Maki:** Rolls made with vinegared rice, seaweed, and various fillings.<br>* **Temaki:** Hand-rolled cone-shaped sushi.<br><br>**Ramen:**<br><br>* **Shoyu Ramen:** Soy sauce-based broth with pork and vegetables.<br>* **Shio Ramen:** Salt-based broth with pork and vegetables.<br>* **Tonkotsu Ramen:** Rich, creamy broth made from pork bones.<br>* **Miso Ramen:** Miso paste-based broth with pork and vegetables.<br><br>**Tempura:**<br><br>* **Tempura Udon:** Thick udon noodles coated in tempura batter and deep-fried.<br>* **Tempura Vegetables:** Lightly battered and deep-fried vegetables.<br><br>**Other Must-Try Dishes:**<br><br>* **Okonomiyaki:** Savory pancake with cabbage, meat, and seafood.<br>* **Yakitori:** Grilled skewers of chicken, pork, and vegetables.<br>* **Takoyaki:** Octopus balls made with batter and filled with octopus, tempura scraps, and mayonnaise.<br>* **Gyoza:** Pan-fried dumplings filled with meat and vegetables.<br>* **Onigiri:** Rice balls wrapped in seaweed.<br>* **Soba:** Buckwheat noodles served cold or hot.<br>* **Udon:** Thick wheat flour noodles served in various broths.<br>* **Curry Rice:** Japanese curry served with rice.<br><br>**Regional Specialties:**<br><br>* **Kyoto:** Kaiseki cuisine, a multi-course meal featuring seasonal ingredients.<br>* **Osaka:** Takoyaki, okonomiyaki, and kushikatsu (deep-fried skewers).<br>* **Hokkaido:** Hokkaido ramen, a unique style of ramen with a rich, creamy broth.<br>* **Nagano:** Soba noodles served with various toppings.<br><br>**Don't Forget:**<br><br>* **Japanese sweets:** Mochi, wagashi, and dorayaki are delicious treats.<br>* **Green tea:** A staple beverage in Japan, enjoyed hot or iced.<br>* **Japanese beer:** Sapporo, Kirin, and Asahi are popular choices.<br><br>**Tips:**<br><br>* **Try local specialties:** Explore regional dishes for a unique culinary experience.<br>* **Visit local markets:** Experience the vibrant atmosphere and sample fresh produce.<br>* **Eat at small restaurants:** Discover hidden gems and authentic Japanese cuisine.<br>* **Be adventurous:** Don't be afraid to try new things and step outside your comfort zone.<br><br><br>Enjoy your culinary journey through Japan!<br><end_of_turn></blockquote></font>

# Load Gemma 2 9B base
* note had to use half-precision to fit into 1 A100 GPU

In [5]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
keras.config.set_floatx("bfloat16")
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_9b_en")

100%|██████████| 779/779 [00:00<00:00, 1.00MB/s]


100%|██████████| 17.2G/17.2G [18:39<00:00, 16.5MB/s]


100%|██████████| 315/315 [00:00<00:00, 385kB/s]


100%|██████████| 4.04M/4.04M [00:01<00:00, 2.32MB/s]


CPU times: user 1min 32s, sys: 52.7 s, total: 2min 25s
Wall time: 19min 28s


# Test a simple query on Gemma 2 9B base

In [8]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 40s, sys: 1.27 s, total: 2min 42s
Wall time: 1min 42s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Japan in a few weeks and I'm wondering what I should eat. I'm not a big fan of sushi, but I'm willing to try it. I'm also not a big fan of raw fish, but I'm willing to try it. I'm also not a big fan of seafood, but I'm willing to try it. I'm also not a big fan of vegetables, but I'm willing to try it. I'm also not a big fan of meat, but I'm willing to try it. I'm also not a big fan of dairy, but I'm willing to try it. I'm also not a big fan of eggs, but I'm willing to try it. I'm also not a big fan of bread, but I'm willing to try it. I'm also not a big fan of pasta, but I'm willing to try it. I'm also not a big fan of rice, but I'm willing to try it. I'm also not a big fan of noodles, but I'm willing to try it. I'm also not a big fan of potatoes, but I'm willing to try it. I'm also not a big fan of beans, but I'm willing to try it. I'm also not a big fan of nuts, but I'm willing to try it. I'm also not a big fan of seeds, but I'm willing to try it. I'm also not a big fan of fruits, but I'm willing to try it. I'm also not a big fan of vegetables, but I'm willing to try it. I'm also not a big fan of meat, but I'm willing to try it. I'm also not a big fan of dairy, but I'm willing to try it. I'm also not a big fan of eggs, but I'm willing to try it. I'm also not a big fan of bread, but I'm willing to try it. I'm also not a big fan of pasta, but I'm willing to try it. I'm also not a big fan of rice, but I'm willing to try it. I'm also not a big fan of noodles, but I'm willing to try it. I'm also not a big fan of potatoes, but I'm willing to try it. I'm also not a big fan of beans, but I'm willing to try it. I'm also not a big fan of nuts, but I'm willing to try it. I'm also not a big fan of seeds, but I'm willing to try it. I'm also not a big fan of fruits, but I'm willing to try it. I'm also not a big fan of vegetables, but I'm willing to try it. I'm also not a big fan of meat, but I'm willing to try it. I'm also not a big fan of dairy, but I'm willing to try it. I'm also not a big fan of eggs, but I'm willing to try it. I'm also not a big fan of bread, but I'm willing to try it. I'm also not a big fan of pasta, but I'm willing to try it. I'm also not a big fan of rice, but I'm willing to try it. I'm also not a big fan of noodles, but I'm willing to try it. I'm also not a big fan of potatoes, but I'm willing to try it. I'm also not a big fan of beans, but I'm willing to try it. I'm also not a big fan of nuts, but I'm willing to try it. I'm also not a big fan of seeds, but I'm willing to try it. I'm also not a big fan of fruits, but I'm willing to try it. I'm also not a big fan of vegetables, but I'm willing to try it. I'm also not a big fan of meat, but I'm willing to try it. I'm also not a big fan of dairy, but I'm willing to try it. I'm also not a big fan of eggs, but I'm willing to try it. I'm also not a big fan of bread, but I'm willing to try it. I'm also not a big fan of pasta, but I'm willing to try it. I'm also not a big fan of rice, but I'm willing to try it. I'm also not a</blockquote></font>

# Load Gemma 2 9B instruct

In [6]:
%%time
# release prevous model's (GPU) memory resources
gemma_lm = None
gc.collect()
# load the new model
keras.config.set_floatx("bfloat16")
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_9b_en")

100%|██████████| 779/779 [00:00<00:00, 975kB/s]


100%|██████████| 17.2G/17.2G [19:37<00:00, 15.7MB/s]


100%|██████████| 315/315 [00:00<00:00, 380kB/s]


100%|██████████| 4.04M/4.04M [00:01<00:00, 2.26MB/s]


CPU times: user 1min 31s, sys: 53.3 s, total: 2min 24s
Wall time: 20min 21s


# Test a simple query on Gemma 2 9B instruct

In [7]:
%%time
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 2min 40s, sys: 1.54 s, total: 2min 42s
Wall time: 1min 40s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I eat in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>Japan is a culinary paradise, offering a wide variety of delicious and unique dishes. Here are some must-try foods when you visit:<br><br>**Sushi & Sashimi:**<br><br>* **Sushi:** Vinegared rice topped with various ingredients, such as seafood, vegetables, and eggs.<br>* **Sashimi:** Thinly sliced raw fish served with soy sauce, wasabi, and pickled ginger.<br>* **Nigiri:** Hand-shaped sushi with a small mound of rice topped with a single ingredient.<br>* **Maki:** Sushi rolls wrapped in seaweed (nori).<br><br>**Ramen:**<br><br>* **Shoyu Ramen:** Ramen with a soy sauce-based broth.<br>* **Shio Ramen:** Ramen with a salt-based broth.<br>* **Miso Ramen:** Ramen with a miso paste-based broth.<br>* **Tonkotsu Ramen:** Ramen with a pork bone broth.<br><br>**Other Popular Dishes:**<br><br>* **Tempura:** Deep-fried seafood and vegetables.<br>* **Udon:** Thick wheat flour noodles served in a hot broth.<br>* **Soba:** Thin buckwheat noodles served cold or hot.<br>* **Yakitori:** Grilled chicken skewers.<br>* **Okonomiyaki:** Savory pancake made with flour, cabbage, and other ingredients.<br>* **Takoyaki:** Octopus balls deep-fried and topped with sauce, mayonnaise, and bonito flakes.<br>* **Curry Rice:** Japanese-style curry served over rice.<br>* **Tonkatsu:** Breaded and deep-fried pork cutlet.<br><br>**Desserts:**<br><br>* **Mochi:** Chewy rice cakes.<br>* **Matcha Ice Cream:** Ice cream flavored with green tea powder.<br>* **Dorayaki:** Two pancakes filled with red bean paste.<br><br>**Drinks:**<br><br>* **Green Tea:** Japan's national drink.<br>* **Sake:** Japanese rice wine.<br>* **Shochu:** Japanese distilled spirit.<br>* **Beer:** Japan has a thriving craft beer scene.<br><br>**Tips:**<br><br>* **Try local specialties:** Each region in Japan has its own unique dishes.<br>* **Eat at counter seats:** This allows you to interact with the chef and see the food being prepared.<br>* **Don't be afraid to ask for recommendations:** Locals are always happy to share their favorite places to eat.<br>* **Be mindful of table manners:** It's considered polite to slurp noodles and make noise when eating.<br>* **Enjoy the experience:** Eating in Japan is a cultural experience as much as it is a culinary one.<br><br><br><end_of_turn></blockquote></font>